# 전국 평균 폭염 일수 추이 선 그래프

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# '평년' 데이터를 제외한 나머지 데이터만 사용
hot_filtered = hot[hot['연도'] != '평년']

# '연도'를 정수형으로 변환
hot_filtered['연도'] = hot_filtered['연도'].astype(int)

# 1974년부터 시작해서 5년씩 묶어주기 (연도 그룹 생성)
hot_filtered = hot_filtered[(hot_filtered['연도'] >= 1974) & (hot_filtered['연도'] <= 2024)]
hot_filtered['연도그룹'] = ((hot_filtered['연도'] - 1974) // 5) * 5 + 1974

# 그룹별 평균 계산
grouped_data = hot_filtered.groupby('연도그룹')['연합계'].mean().reset_index()

# 한글 폰트 설정 (윈도우의 경우 'Malgun Gothic')
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프 크기 설정
plt.figure(figsize=(12, 6))

# 선 그래프 그리기 (5년 단위 평균값)
plt.plot(grouped_data['연도그룹'], grouped_data['연합계'], marker='o', linestyle='-', color='#FF6347', linewidth=2, markersize=8)

# x축과 y축 레이블 및 타이틀 설정
plt.xlabel('연도 그룹 (5년 단위)', fontsize=12, fontweight='bold')
plt.ylabel('연합계', fontsize=12, fontweight='bold')
plt.title('5년 단위 폭염 일수 변화 (1974년 ~ 2024년)', fontsize=16, fontweight='bold')

# x축 레이블 회전 및 폰트 크기 조정
plt.xticks(grouped_data['연도그룹'], rotation=0, fontsize=15)

# 그리드 추가 (점선 스타일)
plt.grid(True, linestyle='--', alpha=0.7)

# 여백 조정
plt.tight_layout()

# 그래프 보여주기
plt.show()

# 세종시 연중 폭염일수 미래 전망

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.font_manager as fm
from mpl_toolkits.axes_grid1 import make_axes_locatable

# 한글 폰트 설정
font_path = 'C:/Windows/Fonts/malgun.ttf'
font_prop = fm.FontProperties(fname=font_path)
plt.rc('font', family=font_prop.get_name())

# SHP 파일 읽기
try:
    shapefile = gpd.read_file('c:\\data\\BND_ADM_DONG_PG\\BND_ADM_DONG_PG.shp', encoding="utf-8")
except UnicodeDecodeError:
    shapefile = gpd.read_file('c:\\data\\BND_ADM_DONG_PG\\BND_ADM_DONG_PG.shp', encoding="cp949")

# 원하는 지역 필터링
shapefile = shapefile.loc[shapefile['ADM_CD'].astype(str).isin([
    '29010110', '29010311', '29010312', '29010330', '29010340', 
    '29010350', '29010360', '29010370', '29010380', '29010390', 
    '29010512', '29010513', '29010514', '29010515', '29010521', 
    '29010522', '29010560', '29010590', '29010600', '29010610', 
    '29010640', '29010660', '29010680', '29010710'
])]

# 온도 데이터 읽기
temperature_data = pd.read_csv('c:\\data\\폭염_열대야일수_세종시\\수정된_파일\\합친파일\\통합_파일.csv')

# 연도 범위를 설정 (2000~2020: 10년 단위, 2021 이후: 20년 단위)
time_periods = {
    '2000~2009': (2000, 2009),
    '2010~2020': (2010, 2020),
    '2021~2040': (2021, 2040),
    '2041~2060': (2041, 2060)
}

# 행정동 이름 정리 함수
def clean_name(name):
    return name.replace(' ', '').replace('세종특별자치시', '').strip()

# shapefile의 행정동 이름 정리
shapefile['ADM_NM_clean'] = shapefile['ADM_NM'].apply(clean_name)

# 각 구간별 폭염일수 평균 계산
avg_heatwave_by_period = {}
for period, (start_year, end_year) in time_periods.items():
    period_data = temperature_data[(temperature_data['연도'] >= start_year) & (temperature_data['연도'] <= end_year)]
    avg_heatwave = period_data.groupby('지역명')['폭염일수'].mean().reset_index()
    avg_heatwave['지역명_clean'] = avg_heatwave['지역명'].apply(clean_name)
    avg_heatwave_by_period[period] = avg_heatwave

# 연기면과 연동면의 폭염일수 값을 가져오기
def get_heatwave_days_for_missing_areas(avg_heatwave):
    yeongi_heatwave_days = avg_heatwave.loc[avg_heatwave['지역명_clean'] == '연기면', '폭염일수'].values[0]
    yeondong_heatwave_days = avg_heatwave.loc[avg_heatwave['지역명_clean'] == '연동면', '폭염일수'].values[0]
    
    # 없는 동에 대해 해당 면의 폭염일수로 대체
    missing_data = pd.DataFrame({
        '지역명_clean': ['나성동', '어진동', '해밀동', '반곡동'],
        '폭염일수': [yeongi_heatwave_days, yeondong_heatwave_days, yeongi_heatwave_days, yeongi_heatwave_days]
    })
    
    # 기존 데이터와 대체 데이터를 합침
    avg_heatwave = pd.concat([avg_heatwave, missing_data], ignore_index=True)
    return avg_heatwave

# 각 구간별로 폭염일수 데이터를 반곡동, 나성동, 해밀동, 어진동에 반영
for period, avg_heatwave in avg_heatwave_by_period.items():
    avg_heatwave_by_period[period] = get_heatwave_days_for_missing_areas(avg_heatwave)

# 시각화를 위한 함수 정의
def plot_heatwave_map(avg_heatwave, period):
    # 정리된 이름을 기준으로 데이터 병합
    merged_data = shapefile.merge(avg_heatwave, left_on='ADM_NM_clean', right_on='지역명_clean', how='left')

    # 불필요한 열 삭제
    merged_data = merged_data.drop(['ADM_NM_clean', '지역명_clean'], axis=1)

    # 폭염일수를 구간으로 나누기 (예: 0-5, 5-10, 10-15일 구간)
    bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65]
    labels = ['0~5일', '5-10일', '10-15일', '15-20일', '20-25일', '25-30일', '30-35일', '35-40일', '40-45일', '45-50일', '50-55일', '55-60일', '60-65일']
    merged_data['폭염일수_구간'] = pd.cut(merged_data['폭염일수'], bins=bins, labels=labels)

    # 시각화
    fig, ax = plt.subplots(figsize=(15, 15))

    # 각 동별로 평균 폭염일수를 구간으로 표현한 지도 생성
    cmap = plt.get_cmap('YlOrRd')
    merged_data.plot(ax=ax, column='폭염일수_구간', cmap=cmap, edgecolor='black', linewidth=0.5)

    # 제목 추가
    plt.title(f'세종시 동별 폭염일수 구간 ({period})', fontsize=16, fontproperties=font_prop)

    # Colorbar 설정
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("bottom", size="5%", pad=0.1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=25))
    sm._A = []
    cbar = plt.colorbar(sm, cax=cax, orientation='horizontal')
    cbar.set_label(f'폭염일수 구간 ({period})', fontsize=12, fontproperties=font_prop)

    # 지도 범위 설정
    ax.set_xlim(merged_data.total_bounds[0], merged_data.total_bounds[2])
    ax.set_ylim(merged_data.total_bounds[1], merged_data.total_bounds[3])

    # 축 제거
    ax.axis('off')

    # 각 동에 폭염일수 구간 텍스트 추가
    for idx, row in merged_data.iterrows():
        if pd.notnull(row['폭염일수']):
            ax.annotate(text=f"{row['ADM_NM']}\n{row['폭염일수']:.1f}", 
                        xy=(row.geometry.centroid.x, row.geometry.centroid.y),
                        xytext=(3, 3), textcoords="offset points", fontsize=10, 
                        fontweight='bold', color='black', ha='center', va='center',
                        fontproperties=font_prop)

    # 지도 표시
    plt.tight_layout()
    plt.show()

# 각 구간별로 시각화
for period, avg_heatwave in avg_heatwave_by_period.items():
    plot_heatwave_map(avg_heatwave, period)


# 세종시 열대야 전망

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정 (Windows: Malgun Gothic, macOS: AppleGothic, Linux: 나눔고딕)
plt.rc('font', family='Malgun Gothic')  # Windows에서 사용
# plt.rc('font', family='AppleGothic')  # macOS에서 사용
# plt.rc('font', family='NanumGothic')  # Linux에서 사용

# 음수 부호 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

# 온도 데이터 읽기
temperature_data = pd.read_csv('C:\\data\\폭염_열대야일수_세종시\\수정된_파일\\합친파일\\통합_파일.csv')

# 5년 단위로 연도 범위 설정
time_periods = {
    2004: (2000, 2004),
    2009: (2005, 2009),
    2014: (2010, 2014),
    2019: (2015, 2019),
    2024: (2020, 2024),
    2029: (2025, 2029),
    2034: (2030, 2034),
    2039: (2035, 2039),
    2044: (2040, 2044),
    2049: (2045, 2049),
    2054: (2050, 2054),
    2059: (2055, 2059)
}

# 각 구간별로 세종시 전체 열대야일수 평균 계산
avg_tropical_nights_by_period = {}

for end_year, (start_year, _) in time_periods.items():
    # 각 구간별 데이터 필터링
    period_data = temperature_data[(temperature_data['연도'] >= start_year) & (temperature_data['연도'] <= end_year)]
    
    # 세종시 전체 평균 열대야일수 계산
    avg_tropical_nights = period_data['열대야일수'].mean()
    
    avg_tropical_nights_by_period[end_year] = avg_tropical_nights

# 시각화
fig, ax = plt.subplots(figsize=(12, 6))

# 선그래프: 열대야일수
ax.plot(list(avg_tropical_nights_by_period.keys()), list(avg_tropical_nights_by_period.values()), color='blue', marker='o', label='열대야일수')

# x축 틱 라벨을 각 구간의 끝 연도로 표시 (회전 없이 가로 방향으로)
ax.set_xticks(list(avg_tropical_nights_by_period.keys()))
ax.set_xticklabels(list(avg_tropical_nights_by_period.keys()), rotation=0)

# 그래프 제목과 축 레이블
ax.set_ylabel('평균 열대야일수', fontsize=12, color='blue')

# 값 표시
for i, (year, value) in enumerate(avg_tropical_nights_by_period.items()):
    ax.text(year, value + 0.5, f"{value:.2f}", ha='center', fontsize=10, color='blue')

# 범례
ax.legend(loc='upper left')

# 그래프 표시
plt.tight_layout()
plt.show()
